### Logistic Regression

#### Deduction
Input: $$
X =
  \begin{bmatrix}
  1 & 2 & \cdots & 3\\
  4 & 4 & \cdots & 6\\
  7 & 8 & \cdots & 9\\
  \vdots & \vdots & \ddots & \vdots \\
  11 & 12& \cdots & 13\\
  \end{bmatrix}_{m*n}
,
Y =
  \begin{bmatrix}
  1\\
  0\\
  1\\
  \vdots\\
  1\\
  \end{bmatrix}_{m*1}
$$

Hypothesis: $$
  H_\theta(x) = g(z) = \frac{1}{1+e^{-z}} = \frac{e^z}{1+e^z}
  = \frac{1}{1+e^{-\theta^T X}} \tag{1}
$$

  
Linear function: $$
  z = \theta^T X, \quad z \in (-\infty, \infty)
$$

Logistic/Sigmoid function: $$
  g(z) = \frac{1}{1+e^{-z}} = \frac{e^z}{1+e^z}, \quad g(z) \in (0, 1)
$$

| z $\geq$ 0 | z < 0 |
| :---: | :---: |
| $\theta^T X$ $\geq$ 0 | $\theta^T X$ < 0 |
| $h_\theta(x)$ $\geq$ 0.5 | $h_\theta(x)$ < 0.5 |
| **g(z)** $\geq$ 0.5 | **g(z)** < 0.5 |
| **y** = 1 | **y** = 0 |

![Sigmoid](./img/sigmoid.png)

```python
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

z = np.linspace(-10, 10, 1000)
g = 1 / (1 + np.e ** -z)

fig, ax = plt.subplots(figsize=(5, 3))
plt.xlabel('z')
plt.ylabel('g(z)')
sns.lineplot(x='z', y='g(z)', ax=ax, data={'z': z, 'g(z)': g})
```

特性：$$
1 - g(z) = 1 - \frac{1}{1+e^{-z}} = \frac{e^{-z}}{1+e^{-z}} = \frac{1}{1+e^z} = g(-z)$$


$H_\theta(x)$ 被Sigmoid函数归一化到(0,1)，表示结果分类结果为True的概率。

$$
\begin{align}
P(y=1|x;\theta) &= H_\theta(x) \\
P(y=0|x;\theta) &= 1 - H_\theta(x)
\end{align}
$$

Assume $y|x;\theta \sim Bernoulli(1, p)$，合并出y的概率质量函数（y在各特定取值上的概率总和为1）:<br> $$
\begin{align}
  P(y|x;\theta) &= p^y (1-p)^{1-y} \\
                &= \begin{cases}
                    p, & \text{y = 1} \\
                    1 - p, & \text{y = 0}
                   \end{cases} \\
                &= (H_\theta(x))^y (1 - H_\theta(x))^{1-y} \tag{2}
\end{align}
$$


Maximum likelihood function是数据集出现概率最大的函数，一个事件已经观察到的维度发生的概率越大，那么未观察到的维度（误差）发生的概率越小。

所有样本的贡献相互独立, 数据集的概率是每一个样本概率之乘积: $$
  L(\theta) = \prod_{j=0}^{m} p(y^j|x^j;\theta)
$$

获得最大化似然的参数, 是对$\theta$的参数估计: $$
\begin{align}
  \theta^* = \operatorname*{arg\ max}_{\theta}\ L(\theta) &= \prod_{j=0}^{m} p(y^j|x^j;\theta) \\
  &= \prod_{j=0}^{m} (H_\theta(x)^j)^y \, (1-H_\theta(x)^j)^{1-y} \tag{3}
\end{align}
$$


Log transformation，1.连乘变成连加方便求导，2.概率很小时候连乘容易等于0: $$
\begin{align}
  \ell(\theta) &= \log(L(\theta)) \\
  &= \sum_{j=0}^{m} y^j\ log(H_\theta(x)^j) + (1-y^j)\ log(1-H_\theta(x)^j) \tag{4} \\
\end{align}
$$

Binary cross entropy(minimize, 预测分布越接近真实分布): $$
  -\ell(\theta) = -\sum_{j=0}^{m} y^j\ log(H_\theta(x)^j) + (1-y^j)\ log(1-H_\theta(x)^j)
$$

Partial Derivative for every $\theta$: $$
\begin{align}
  \frac{\partial \ell(\theta)}{\partial \theta_i} &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} + (1-y^j) \frac{0-1}{1-H_\theta(x)^j}) \frac{\partial}{\partial \theta_i} H_\theta(x)^j \\
        &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} - (1-y^j) \frac{1}{1-H_\theta(x)^j}) \frac{\partial}{\partial \theta_i} H_\theta(x)^j \\
  \because g'(z) &= \frac{e^{-z}}{(1+e^{-z})^2} \\
        &= \frac{1}{1+e^{-z}} \cdot \frac{e^{-z}}{1+e^{-z}} \\
        &= g(z) \cdot (1-g(z)) \\
  \therefore \frac{\partial \ell(\theta)}{\partial \theta_i} &= \sum_{j=0}^{m} (y^j \frac{1}{H_\theta(x)^j} - (1-y^j) \frac{1}{1-H_\theta(x)^j})\ H_\theta(x)^j\ (1-H_\theta(x)^j)\ x_i^j \\
  &= \sum_{j=0}^{m} (y^j(1-H_\theta(x)^j) - (1-y^j)H_\theta(x)^j)\ x_i^j \\
  &= \sum_{j=0}^{m} (y^j - H_\theta(x)^j)\ x_i^j \tag{5}
\end{align}
$$

Gradient ascent to optimize $\theta$: $$
\begin{align}
  \theta_i &= \theta_i + \frac{\partial log(L(\theta))}{\partial \theta_i} \\
           &= \theta_i + \frac{\partial \ell(\theta)}{\partial \theta_i} \\
           &= \theta_i + \sum\limits_{j=0}^m (y^j - H_\theta(x)^j) x_i^j
\end{align}
$$

Gradient descent: $$
\theta_i = \theta_i - \sum\limits_{j=0}^m (H_\theta(x)^j - y^j) x_i^j \tag{6}
$$

#### Analysis

1. 设定阈值。Logistic regression 计算出的是一个类似概率的值，不一定大于0.5就判断为正，可以根据实际情况设定。
2. 置信度，除了预测出分类，还能给出分类结果的置信度。
3. 对数据倾斜敏感，正负数据比相差悬殊时预测效果不好。损失函数二值交叉熵越小，代表预测分布越接近真实分布，预测越准确。最大似然估计，是所有样本被预测正确的最大概率。样本集正例只有几个，几乎都是负例，则很容易预测准确率很高，但正例预测对的概率很低，此时要对数据重采样/欠采样。

#### Logistic Function
现实生活中经常要探索事件A发生概率P和因素X=(X_1,X_2,...,X_n)之间的关系，线性回归用于逻辑回归有两个问题：
1. 取值范围不同；
2. 自变量很小或者很大时，对因变量的影响小，自变量达到中间某个阈值时，对因变量影响大。

##### 对取值范围问题：
因变量和自变量之间的多元线性关系：$Y = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n, \quad \sum\limits_{i=0}^n \theta_i x_i \in (-\infty, +\infty) $

想利用线性回归模型演化出逻辑回归，要把$(-\infty, +\infty)$的范围归一化到(0,1)上，对数函数是把(0,1)的自变量映射到$(-\infty, 0)$上，$log(Y) = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n$，这时如果$Y \in (0,1)，log(Y) \in (-\infty, 0)$，还少一半对称的范围。

![Log](./img/log.png)
```python
import numpy as np
from bokeh.plotting import figure, show
x = np.linspace(-10, 10, 1000)
y = np.log(x)
p = figure(title="log(x)", x_axis_label='x', x_range=(-10,10), y_range=(-2,2), plot_height=300)
p.line(x, y, line_width=2)
show(p)
```

几率（odd）是事件发生与不发生概率的比，假设事件E发生的概率为p：$$
odd(E) = \frac{p}{1-p} = \frac{1}{\frac{1}{p}-1} \\
  \because p \in (0,1) \\
  \therefore \frac{p}{1-p} \in (0, +\infty) => log(\frac{p}{1-p}) \in (-\infty, +\infty)
$$

![Odd](./img/odd.png)
![Logit](./img/logit.png)

得出logit变换：$$log(\frac{p}{1-p}) = \theta_0 x_0 + \theta_1 x_1 + ... + \theta_n x_n = \sum\limits_{i=0}^n \theta_i x_i \tag{7}
$$
log是指数变换，it是odd函数，logit变换就是对odd函数做指数变换。

对logit变换函数求反函数，得出logistic function：$$
\frac{p}{1-p} = e^{\theta^T X} \\
p = \frac{e^{\theta^T X}}{1+e^{\theta^T X}}
$$

##### 对自变量对因变量特定区间内影响大小问题 $^{[1]}$：
X需要发生很大的变化才引起p的微弱改变，比如，杀虫剂使用剂量X的情况下，杀死害虫的概率P。

构造p的函数f(p)，使得p在0、1附近发生微小的变化，对应f(p)发生较大的变化，同时f(p)尽可能简单，$$
\frac{\partial{f(p)}}{\partial p} = \frac{1}{p} + \frac{1}{1 - p}
$$

![](./img/logit_init.png)

对P求积分，得到logit变换，$$
f(p) = ln(\frac{p}{1-p})
= \sum\limits_{i=0}^n \theta_i x_i
$$
logit变换最初使用在人口增长模型中$^{[2]}$

##### Sigmoid的激活函数特性
Sigmoid函数是神经网络中的激活函数，不常用，因为Sigmoid函数在因变量接近0或1时，曲线非常平滑，导数几乎为0，导致梯度下降收敛非常缓慢，梯度消失，Sigmoid进入饱和区间中。

多层神经网络中，反向传播时，随着网络层次增加，各个导数相乘，越靠近输入层的乘积越接近0，参数更新的幅度越小，速率越慢，有实验发现含有四个隐藏层的网络结构中，第一层比第四层慢了接近100倍。多层神经网络中，Sigmoid更容易饱和，网络就几乎不学习。三层神经网络（两层激活函数）的数学模型（没有截距和正则化）：

Forward propagation: $$
\begin{align}
z_1 &= w_{11} x_1 + w_{12} x_2 \\
H_1 &= g(z_1) \\
z_2 &= w_{21} h_{11} + w_{22} h_{12} \\
H_2 &= g(z_2) \\
z_3 &= w_{31} h_{21} + w_{32} h_{22} \\
L &= \ell(z_3, y) \\
\end{align}
$$

Back-propagation: $$
\begin{align}
\frac{\partial L}{\partial w_{11}} = \left(\frac{\partial L}{\partial z_3} \cdot \frac{\partial z_3}{\partial h_{21}} \cdot \frac{\partial h_{21}}{\partial z_2} \cdot \frac{\partial z_2}{\partial h_{11}} \cdot \frac{\partial h_{11}}{\partial z_1} \cdot \frac{\partial z_1}{\partial w_{11}}, \\
\frac{\partial L}{\partial z_3} \cdot \frac{\partial z_3}{\partial h_{21}} \cdot \frac{\partial h_{21}}{\partial z_2} \cdot \frac{\partial z_2}{\partial h_{12}} \cdot \frac{\partial h_{12}}{\partial z_1} \cdot \frac{\partial z_1}{\partial w_{11}}, \\
\frac{\partial L}{\partial z_3} \cdot \frac{\partial z_3}{\partial h_{22}} \cdot \frac{\partial h_{22}}{\partial z_2} \cdot \frac{\partial z_2}{\partial h_{11}} \cdot \frac{\partial h_{11}}{\partial z_1} \cdot \frac{\partial z_1}{\partial w_{11}}, \\
\frac{\partial L}{\partial z_3} \cdot \frac{\partial z_3}{\partial h_{22}} \cdot \frac{\partial h_{22}}{\partial z_2} \cdot \frac{\partial z_2}{\partial h_{12}} \cdot \frac{\partial h_{12}}{\partial z_1} \cdot \frac{\partial z_1}{\partial w_{11}} \right)
\end{align}
$$


反向传播中这几项 $\left(\frac{\partial h_{21}}{\partial z_2} \cdot \frac{\partial h_{11}}{\partial z_1} \right)$ 是Sigmoid函数的导数，所以越乘越小。

如果权重值初始化过大，Sigmoid函数值越接近于0或1，导数越接近0，导致大多数神经元将饱和，权重矩阵的初始化必须特别留意。

Sigmoid函数值在(0, 1)之间，非zero-centered，对不同参数的迭代方向相反，使得收敛时梯度下降走Z字型路线。$^{[3]}$

深度学习中一般以梯度下降更新参数，$$
\omega \leftarrow \omega - \eta \cdot \frac{\partial L}{\partial \omega}
$$

对于参数$\omega_i$来说，$$
\frac{\partial L}{\partial \omega_i} = \frac{\partial L}{\partial g} \frac{\partial g}{\partial z} \frac{\partial z}{\partial \omega_i} = x_i \cdot \frac{\partial L}{\partial g} \frac{\partial g}{\partial z}
$$

参数更新为，$$
\omega_i \leftarrow \omega_i - \eta \cdot x_i \cdot \frac{\partial L}{\partial g} \frac{\partial g}{\partial z}
$$

$\eta$是模型超参数学习率，$\frac{\partial L}{\partial g} \frac{\partial g}{\partial z}$是固定函数的导数，所以$\omega_i$更新方向是由$x_i$决定的。

我们以二维的情况为例，参数$\omega_0, \omega_1$的最优解$\omega_0^*, \omega_1^*$，优化目标分别是$$
\begin{cases}  \omega_0 < \omega_0^* \\
\omega_1 \ge \omega_1^*
\end{cases}
$$
需要$\omega_0$增大，$\omega_1$减小，那就要求$x_0, x_1$符号相反。上一级神经元采用Sigmoid函数作为激活函数，输出中间层H总是正数，无法做到$x_0, x_1$（即$h_{10}, h_{11}$）符号相反。模型为了收敛不得不走Z字型。优化目标在南边，优化参数一个往东一个往西，实际中迭代更新时即使超参数学习率设置比较大，收敛也非常缓慢，数值变化很小，示意图如下。![z_gradient_descent](./img/z_gradient_descent.png)

#### Maximum likelihood function $^{[4]}$
概率是某事件发生之前，根据环境参数对其发生可能性的预测。比如抛硬币之前，根据硬币是密度均匀分布推测正反面概率都是50%。

似然是某事件发生后，根据确定的结果推测产生结果的环境参数。比如抛1000次硬币，496次朝上，504次朝下，可以大致推断出硬币密度分布均匀，抛硬币的人没有作弊。

如果用$\theta$表示环境参数，x表示结果，概率可表示为，$P(x|\theta)$，似然可以表示为，$L(\theta|x)$.

最大似然估计函数是样本的函数，根据样本的观测值$x_1,x_2,...x_n,$带入统计表达式，得到最大似然估计值。

### Cross entropy
**自信息**是事件发生前事件发生的不确定性，是事件发生后所包含的信息量，也是消除不确定性所需的信息量。

**信息量**是单个事件不确定性的度量，和事件发生概率大小成反比。对于已经发生的事件i，发生概率是$p_i$，信息量：$I(p_i) = -log_2(p_i)$

**信息熵**是随机变量不确定性的度量，所有可能发生事件的信息量的期望。
$$H(X) = -\sum_{i=1}^n p(x_i) log(p(x_i))$$

有一个骰子只有ABCD四个面，两头是尖的用来转动，四个面以相等的概率$1/4$出现，如果事件是A面朝上，信息量$I = -log_2(1/4) = 2$，占用2bit的信息。

掷骰子可能产生四种情况，信息熵的计算，$H = (-1/4 * log_2(1/4)) * 4 = 2$。

如果骰子密度分布不均匀，ABCD各面朝上概率分别为1/2, 1/4, 1/8, 1/8，则信息熵：
$$H = - 1/2*log_2(1/2) - 1/4*log_2(1/4) - 1/8*log_2(1/8) - 1/8*log_2(1/8) = 1.75$$

可见，可能发生n个事件的概率都一致，都为1/n，信息熵最大。

连续随机变量x，其概率密度函数p(x)，信息熵定义$^{[5]}$： $$H(X) = \int_x p(x)I(x)dx = -\int_x p(x)log(p(x))dx$$

**交叉熵**是衡量在真实分布下，使用非真实分布能够消除系统不确定性的大小。
$$H(p, q) = -\sum_{i=1}^n p(x_i) log(q(x_i))$$
p(x)是事件真实分布下的概率，q(x)是事件非真实分布下的概率。在二值交叉熵中q(x) = 1 - p(x)。衡量log(q(x))在p(x)分布下的期望。

掷骰子游戏中$^{[6]}$，四个面以相等的概率$1/4$出现，掷出后用碗盖住骰子，使看不到结果，假如结果是A面朝上，我们需要猜两次才能确定是A，第一次问是AB吗，或者是CD吗？第二次问是A吗，或者是B吗？我们猜测时用的四个面的概率分布和真实概率分布相同，交叉熵是我们猜测次数的期望，等于信息熵。
$$H(p,q) = (-1/4 * log_2(1/4)) * 4 = (1/4 * 2次) * 4 = 2次$$

如果四个面ABCD各面朝上概率分别为1/2, 1/4, 1/8, 1/8，但庄家作弊没有告诉我们，我们只能假设四个面等概率向上，真实概率分布和我们猜测用的概率分布不同，$$
\begin{align}
H(p,q) &= 1/2*log_2(4) + 1/4*log_2(4) + 1/8*log_2(4) + 1/8*log_2(4) \\
&= 1/2*2 + 1/4*2 + 1/8*2 + 1/8*2 \\
&= 2
\end{align}
$$
知道真实概率分布而进行猜测的交叉熵，等于信息熵，等于1.75。

可见，**_交叉熵越低，假设的概率分布越接近真实概率分布，当交叉熵等于信息熵时，假设概率分布和真实概率分布重合。_**

连续随机变量x，其概率密度函数p(x)，假设分布概率密度函数q(x)，相对熵定义： $$H(p,q) = -\int_x p(x)log(q(x))dx$$

### References
[1] [如何理解logistic函数?](https://www.zhihu.com/question/36714044)

[2] [Logistic Equation](https://mathworld.wolfram.com/LogisticEquation.html)

[3] [谈谈激活函数以零为中心的问题](https://liam.page/2018/04/17/zero-centered-active-function/)

[4] [最大似然估计、n阶矩、协方差（矩阵）、（多元）高斯分布 学习摘要](https://www.cnblogs.com/boceng/p/11358480.html)

[5] [机器学习中各种熵的定义及理解](https://www.cnblogs.com/wkang/p/10068475.html)

[6] [玩个游戏来理解交叉熵](https://mp.weixin.qq.com/s/8mlqF9d9rO5arENMaJyZ-Q)